# Mobile games development

## Задание

##### Вам предстоит поработать аналитиком данных в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:
##### 1) Расчётать Retention (по дням от даты регистрации);
##### 2) На основе данных, полученных в ходе проведения A/B теста, определить, какой набор акционных предоложений можно считать лучшим и на основе каких метрик стоит принять правильное решение;
##### 3) Предложить метрики для оценки результатов последнего проведенного тематического события в игре.

## Данные

##### Расчёр Retention
##### reg_data.csv – данные о времени регистрации (таблица из 2 колонок):
##### 1) reg_ts - время регистрации в формате Unix;
##### 2) uid - id пользователя;
##### auth_data.csv – данные о времени захода пользователей в игру (таблица из 2 колонок):
##### 1) auth_ts - время повторного входа в игру в формате Unix;
##### 2) uid - id пользователя.
##### Определение набора акционных предложений
##### sets_sahres.csv - данные о выручке и группах (таблица из 3 колонок):
##### 1) user_id - id пользователя;
##### 2) revenue - выручка;
##### 3) testgroup - вид группы (a - контрольная, b - тестовая).

In [2]:
import pandas as pd
import numpy as np
from operator import attrgetter

task1_reg = pd.read_csv('reg_data.csv', sep=';')
task1_auth = pd.read_csv('auth_data.csv', sep=';')

FileNotFoundError: [Errno 2] No such file or directory: 'reg_data.csv'

In [ ]:
task1_reg.head()

In [ ]:
task1_reg.tail()

In [ ]:
task1_reg.shape

In [ ]:
task1_reg.isna().sum()

In [ ]:
task1_auth.head()

In [ ]:
task1_auth.tail()

In [ ]:
task1_auth.shape

In [ ]:
task1_auth.isna().sum()

In [ ]:
# Данные о повторном входе в приложение содержат данные о регистрации,
# поэтому дальше использую только второй датафрейм (task1_auth)

### Расчет Retention

In [3]:
def retention(data):
    """
    Перевожу данные о времени входа в приложение из формата unix

    Извлекаю день и месяц входа
    
    Присваиваю когорты на основе данных о первом входе в приложение для каждого пользователя
    
    Рассчитываю номер месячного периода (делю на 30)
    
    Агрегирую данные по когортам и рассчитанному номеру месячного периода
    
    Создаю сводную таблицу для когортного анализа
    
    Рассчитываю размеры когорт (первый столбец сводной таблицы)
    
    Вычисляю коэффициенты удержания (делю на размер когорты)
    """

    data.auth_ts = pd.to_datetime(data.auth_ts, unit='s')
    
    data['order_period'] = data.auth_ts.dt.to_period('M')
    data['order_period_day'] = data.auth_ts.dt.to_period('D')
    
    data['cohort'] = data.groupby('uid').\
                          auth_ts.transform('min').\
                          dt.to_period('M')
    data['cohort_day'] = data.groupby('uid').\
                              auth_ts.transform('min').\
                              dt.to_period('D')
    
    data['period_number_month'] = np.floor((data.order_period_day - data.cohort_day).apply(attrgetter('n')) / 30)
    
    df_chohort = data.groupby(['cohort', 'period_number_month']).\
                      agg(n_customers=('uid', 'nunique')).\
                      reset_index()
    
    cohort_pivot = df_chohort.pivot_table(index='cohort', columns='period_number_month', values='n_customers')
    
    cohort_size = cohort_pivot.iloc[:, 0]
    
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0)
    
    
    return retention_matrix

In [4]:
retention(task1_auth)

NameError: name 'task1_auth' is not defined

### Определение набора акционных предложений
##### Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.
##### Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [4]:
task2 = pd.read_csv('sets_shares.csv', sep=';')

In [74]:
# Конверсия в платящего пользователя (CR) - процент платящих пользователей
# CR = количество платящих пользователей / общее количество пользователей

control_cr = 1928 / 202103
test_cr = 1805 / 202667

control_cr > test_cr # True, следовательно акционные предложения в тестовой группе менее привлекательны

# Средниц доход на платящего пользователя (ARPPU)
# ARPPU = общий доход / количество платящих пользователей

# Разбиваю датафрейм на 2 (данные о контрольной и тестовой группах)
control = task2.query('testgroup=="a"')
test = task2.query('testgroup=="b"')

# Нахожу общие доходы групу
ctrl_com_rev = sum(control.revenue)
tst_com_rev = sum(test.revenue)

# Считаю количество платящих пользователей каждой группы
ctrl_pay_users = control.query('revenue!=0').\
                         revenue.count()
test_pay_users = test.query('revenue!=0').\
                      revenue.count()

# Нахожу ARPPU каждой группы
ctrl_arppu = ctrl_com_rev / ctrl_pay_users # 2663.9984439834025
test_arppu = tst_com_rev / test_pay_users # 3003.6581717451522

diff = ctrl_arppu - test_arppu # 339.65972776174976

# Разница сильно велика, поэтому делаю вывод о том, что тестовый набор предложений лучше

### Метрики для оценки события
##### В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?
##### Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

In [9]:
# Метрики для обычного события

# Количество новых игроков во время проведения события
# Среднее время игрового сеасна игроков
# Средний прогресс игроков в событии (уровень после завершения события)
# Количество игроков, которые получили все награды, прошли событие до конца
# ARPU
# ARPPU
# Конверсия в платящего пользователя
# Retention


# Метрики для события с усложненной механикой (остаются прошлые метрики и добавляются новые):

# Среднее количетсво откатов игроков
# Среднее количество уровней, на которые откатились игроки
# Уровень фрустрации (процент оттока после отката уровней)
# ARPPU откатившихся игроков